### Imports Librerías

In [1]:
# TF 2.13
import tensorflow as tf
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from typing import Tuple, Dict
import numpy as np
import math

import tensorflow.keras.applications.efficientnet_v2 as effnV2
from tensorflow.keras.layers import Dense, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Dropout
from sklearn.utils.class_weight import compute_class_weight

2023-11-16 13:26:47.858446: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 13:26:47.858506: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 13:26:47.858535: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 13:26:47.869041: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Definición metodos creación del dataset

In [2]:
dict_map_class = {
    'Loose Silky-bent': 0,
    'Common Chickweed': 1,
    'Scentless Mayweed': 2,
    'Small-flowered Cranesbill': 3,
    'Fat Hen': 4,
    'Charlock': 5,
    'Sugar beet': 6,
    'Cleavers': 7,
    'Black-grass': 8,
    'Shepherds Purse': 9,
    'Common wheat': 10,
    'Maize': 11,   
}

dict_map_class_inverted = {v: k for k, v in dict_map_class.items()}

def get_dict_dataset(
    dataset_path: str = 'dataset'
):
    """
    Creates a dictionary mapping image paths to tuples containing class and partition.

    Args:
        dataset_path (str): Path to the dataset directory. Defaults to 'dataset'.

    Returns:
        dict: A dictionary where keys are image paths and values are tuples (class, partition).
    """

    dict_dataset = {}

    train_classes = os.listdir(os.path.join(dataset_path, 'train'))

    for train_class in train_classes:
        class_path = os.path.join(dataset_path, 'train', train_class)
        train_imgs = os.listdir(class_path)
        
        # Split Train images to a 80% for a Train Split for each class
        for train_img in train_imgs[:int(len(train_imgs)*0.8)]:
            train_img_path = os.path.join(class_path, train_img)
            dict_dataset[train_img_path] = (dict_map_class[train_class], 'Train')

        # Assign the rest 20% to Valid Split for each class
        for valid_img in train_imgs[int(len(train_imgs)*0.8):]:
            valid_img_path = os.path.join(class_path, valid_img)
            dict_dataset[valid_img_path] = (dict_map_class[train_class], 'Valid')

    # Geting Test Images
    test_path = os.path.join(dataset_path, 'test')
    test_imgs = os.listdir(test_path)

    for test_img in test_imgs:
        test_img_path = os.path.join(test_path, test_img)
        dict_dataset[test_img_path] = ("Unkown", 'Test')

    return dict_dataset


def dict2dataframe(
    input_dict: Dict[str, Tuple]
):
    """
    Converts a dictionary to a pandas DataFrame with columns for 'path', 'label', and 'split'.

    Args:
        input_dict (dict): A dictionary where keys are image paths and values are tuples (label, split).

    Returns:
        pd.DataFrame: A DataFrame with columns 'path', 'label', and 'split'.
    """
    df = pd.DataFrame([(key, values[0], values[1]) for key, values in input_dict.items()], columns=['path', 'label', 'split'])

    # Returns shuffled datasets
    return df.sample(frac=1, random_state=42).reset_index(drop=True)


### Definición modelos prueba

In [3]:
def load_efficienNetV2(
    num_classes = int,
):
    """
    Loads EfficientNetV2 model with customized top layers for a specific number of classes.

    Args:
        num_classes (int): Number of classes for the final classification layer.

    Returns:
        tf.keras.models.Model: An instance of the EfficientNetV2 model with customized top layers.
    """
    redEffi = effnV2.EfficientNetV2B0(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3),
    )

    x = redEffi.outputs[0]
    x_mean = GlobalAveragePooling2D()(x)
    x_max = GlobalMaxPooling2D()(x)
    x = Concatenate()([x_mean, x_max])
    x = Dropout(0.3)(x)

    x = Dense(num_classes, activation = 'softmax')(x)

    model = tf.keras.models.Model(
            inputs=redEffi.inputs, outputs=[x])

    return model

### Creacion TF Datasets

In [4]:
def tf_augmenter():
    """
    Returns a TensorFlow function for data augmentation.

    The returned function applies random transformations to the input images,
    including random flips (up-down and left-right), random brightness adjustments,
    and random contrast adjustments.

    Returns:
        callable: A TensorFlow function that takes a variable number of arguments (tensors)
                  representing the dataset and applies data augmentation to the images.
    """
    @tf.function
    def f(*dataset):
        output= list(dataset)
        image = output[0]
        
        if tf.random.uniform([1], minval=0, maxval=1) > 0.5:
            image = tf.image.random_flip_up_down(image)
        if tf.random.uniform([1], minval=0, maxval=1) > 0.5:
            image = tf.image.random_flip_left_right(image)
        if tf.random.uniform([1], minval=0, maxval=1) > 0.5:
            image = tf.image.random_brightness(image, 0.15)
        if tf.random.uniform([1], minval=0, maxval=1) > 0.7:
            image = tf.image.random_contrast(image, 0.6, 1.4)

        output[0] = image
        return output
    return f


@tf.function
def load_image(*inputs):
    """
    TensorFlow function to load an image using a numpy function.

    Args:
        *inputs: Variable number of input tensors.

    Returns:
        list: A list of output tensors with the loaded image as the first element.
    """
    outputs = list(inputs)
    image = tf.numpy_function(load_image_np, [inputs[0]], tf.float32)
    image.set_shape([None, None, 3])
    outputs[0] = image
    
    return outputs


def load_image_np(path):
    """
    Loads an image from the specified path and convert it to a NumPy array.

    Args:
        path (str): The path to the image file.

    Returns:
        np.ndarray: A NumPy array representing the loaded image in RGB format.
    """
    return np.array(Image.open(path).convert('RGB')).astype(np.float32)


def resize(index=0, resize_to=None):
    """
    Returns a TensorFlow function to resize an image in a dataset.

    Args:
        index (int): Index of the image tensor in the dataset. Defaults to 0.
        resize_to (tuple or list or None): Target size for resizing. If None, no resizing is performed. Defaults to None.

    Returns:
        callable: A TensorFlow function that resizes the image in the dataset.
    """
    def f(*dataset):
        output = list(dataset)
        resized_image = tf.image.resize(dataset[index], resize_to)
        resized_image = tf.cast(resized_image, tf.uint8)
        output[index] = resized_image
        
        return output
    return f


def preprocess_input(index):
    """
    Returns a TensorFlow function to preprocess an image in a dataset.

    Args:
        index (int): Index of the image tensor in the dataset.

    Returns:
        callable: A TensorFlow function that preprocesses the image in the dataset.
    """
    @tf.function
    def f(*dataset):
        output = list(dataset)
        image = dataset[index]
        image = tf.cast(image, tf.float32)
        image = image / 255.
        output[index] = image
        
        return output
    return f

In [5]:
def get_dataset(
    df: pd.DataFrame,
    input_size: Tuple[int, int],
    shuffle: bool = False,
    batch_size: int = None,
    gray_scale: bool = False,
    augmenter: bool = False,
    num_aug: int = None,
    test_set: bool = False
)->tf.data.Dataset:
    """
    Creates a TensorFlow dataset from a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing information about the dataset.
        input_size (Tuple[int, int]): Tuple representing the target size for image resizing.
        shuffle (bool): Whether to shuffle the dataset. Defaults to False.
        batch_size (int): Batch size for the dataset. If None, no batching is performed. Defaults to None.
        gray_scale (bool): Whether to convert images to grayscale. Defaults to False.
        augmenter (bool): Whether to apply data augmentation. Defaults to False.
        num_aug (int): Number of augmentations to apply if augmenter is True. Defaults to None.
        test_set (bool): Whether the dataset is a test set. Defaults to False.

    Returns:
        tf.data.Dataset: A TensorFlow dataset prepared based on the provided options.
    """
    # Prints info about labels distribution 
    print('Number of instances per label: ',
          pd.Series(df['label']).value_counts(), sep='\n')
    print('\nPercentaje of instances per label: ',
          pd.Series(df['label']).value_counts().div(pd.Series(df['label']).shape[0]),
          sep='\n')

    names = np.array(df['path'], dtype=str)

    if not test_set:
        labels = np.array(tf.keras.utils.to_categorical(df['label'], num_classes=12))
    else:
        labels = np.ones(len(names))

    data = names, labels

    # Creates a tf dataset from paths and labels
    dataset = tf.data.Dataset.from_tensor_slices(data)

    # Shuffles the entire dataset
    if shuffle:
        print(' > Shuffle')
        dataset = dataset.shuffle(len(names))

    # Loading Images
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Resize to desired size
    dataset = dataset.map(resize(resize_to=input_size), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Augmentation
    if augmenter:
        print(f' > Augmentamos datos numero {num_aug}')
        if num_aug == 1:
            dataset = dataset.map(tf_augmenter(), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Preprocessing input
    dataset = dataset.map(preprocess_input(0), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Converts to gray Scale
    if gray_scale:
        print(' > Escala de grises')
        dataset = dataset.map(lambda *args: (tf.image.rgb_to_grayscale(args[0]), *args[1:]))

    # Prepare batch_size
    if batch_size is not None:
        print(' > Establecemos el batchsize')
        dataset = dataset.batch(batch_size)
    
    # Prefetch to overlap data preprocessing and model execution
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

## **Creamos los DataFrames con el contenido de nuestro dataset**

In [6]:
dict_dataset = get_dict_dataset()

# Creamos el DataFrame 
df_dataset = dict2dataframe(dict_dataset)
df_dataset

,path,label,split
0,dataset/train/Scentless Mayweed/8c496e84a.png,2,Train
1,dataset/train/Common wheat/4a56f32c6.png,10,Train
2,dataset/test/99569b224.png,Unkown,Test
3,dataset/train/Black-grass/e0380dff9.png,8,Valid
4,dataset/train/Fat Hen/9708f9c0e.png,4,Train
...,...,...,...
5539,dataset/train/Shepherds Purse/953ced7c6.png,9,Train
5540,dataset/test/8cf909eb3.png,Unkown,Test
5541,dataset/test/98062cd87.png,Unkown,Test
5542,dataset/test/cc3d2a59a.png,Unkown,Test


In [7]:
df_dataset['label'].value_counts()

label
Unkown    794
0         654
1         611
2         516
3         496
4         475
5         390
6         385
7         287
8         263
9         231
10        221
11        221
Name: count, dtype: int64

In [8]:
df_dataset_train = df_dataset[df_dataset['split'] == 'Train'].drop(columns=['split'])
df_dataset_train

,path,label
0,dataset/train/Scentless Mayweed/8c496e84a.png,2
1,dataset/train/Common wheat/4a56f32c6.png,10
4,dataset/train/Fat Hen/9708f9c0e.png,4
5,dataset/train/Sugar beet/6d623072a.png,6
8,dataset/train/Small-flowered Cranesbill/869252...,3
...,...,...
5534,dataset/train/Scentless Mayweed/948251df3.png,2
5535,dataset/train/Sugar beet/29a0e6bf9.png,6
5537,dataset/train/Charlock/8b35222d0.png,5
5539,dataset/train/Shepherds Purse/953ced7c6.png,9


In [9]:
df_dataset_train['label'].value_counts()

label
0     523
1     488
2     412
3     396
4     380
5     312
6     308
7     229
8     210
9     184
10    176
11    176
Name: count, dtype: int64

In [10]:
df_dataset_valid = df_dataset[df_dataset['split'] == 'Valid'].drop(columns=['split'])
df_dataset_valid

,path,label
3,dataset/train/Black-grass/e0380dff9.png,8
10,dataset/train/Scentless Mayweed/d748c7307.png,2
15,dataset/train/Fat Hen/e6b756e98.png,4
22,dataset/train/Small-flowered Cranesbill/ecf58a...,3
24,dataset/train/Charlock/d1b362c43.png,5
...,...,...
5500,dataset/train/Loose Silky-bent/fc2b27fff.png,0
5501,dataset/train/Charlock/fc3e15a2e.png,5
5503,dataset/train/Scentless Mayweed/d1e775b97.png,2
5505,dataset/train/Common Chickweed/f50c8181a.png,1


In [11]:
df_dataset_valid['label'].value_counts()

label
0     131
1     123
2     104
3     100
4      95
5      78
6      77
7      58
8      53
9      47
11     45
10     45
Name: count, dtype: int64

In [12]:
df_dataset_test = df_dataset[df_dataset['split'] == 'Test'].drop(columns=['split'])
df_dataset_test

,path,label
2,dataset/test/99569b224.png,Unkown
6,dataset/test/d14aa43f3.png,Unkown
7,dataset/test/b47691c08.png,Unkown
12,dataset/test/19fdf19fb.png,Unkown
14,dataset/test/7d4cd07ad.png,Unkown
...,...,...
5526,dataset/test/b30ab4659.png,Unkown
5536,dataset/test/bb1c84bbc.png,Unkown
5540,dataset/test/8cf909eb3.png,Unkown
5541,dataset/test/98062cd87.png,Unkown


In [13]:
df_dataset_test['label'].value_counts()

label
Unkown    794
Name: count, dtype: int64

### Visualizamos muestras random del dataset

In [ ]:
fig = plt.figure(figsize = (20, 5))

for i in range(20):
    img = Image.open(df_dataset_train.iloc[i]['path'])
    ax = fig.add_subplot(2, 10, i + 1)
    ax.imshow(img.convert('RGB'))
    title = dict_map_class_inverted[df_dataset_train.iloc[i]['label']]
    ax.set_title(title, fontsize=10)
    ax.axis('off')

## **Creamos los datasets para train, val y test**

In [14]:
train_tfdataset = get_dataset(
    df=df_dataset_train,
    input_size=(224,224),
    batch_size=64,
    shuffle=True,
    gray_scale=False,
    augmenter=True,
    num_aug=1,
)

train_tfdataset

Number of instances per label: 
label
0     523
1     488
2     412
3     396
4     380
5     312
6     308
7     229
8     210
9     184
10    176
11    176
Name: count, dtype: int64

Percentaje of instances per label: 
label
0     0.137849
1     0.128624
2     0.108593
3     0.104375
4     0.100158
5     0.082235
6     0.081181
7     0.060358
8     0.055351
9     0.048498
10    0.046389
11    0.046389
Name: count, dtype: float64


2023-11-16 13:27:16.081126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 13:27:16.088069: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 13:27:16.088499: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 13:27:16.091691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 13:27:16.092049: I tensorflow/compile

 > Shuffle
 > Augmentamos datos numero 1
 > Establecemos el batchsize


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 12), dtype=tf.float32, name=None))>

In [15]:
valid_tfdataset = get_dataset(
    df=df_dataset_valid,
    input_size=(224,224),
    batch_size=64,
    shuffle=True,
    gray_scale=False,
    augmenter=False,
)

valid_tfdataset

Number of instances per label: 
label
0     131
1     123
2     104
3     100
4      95
5      78
6      77
7      58
8      53
9      47
11     45
10     45
Name: count, dtype: int64

Percentaje of instances per label: 
label
0     0.137029
1     0.128661
2     0.108787
3     0.104603
4     0.099372
5     0.081590
6     0.080544
7     0.060669
8     0.055439
9     0.049163
11    0.047071
10    0.047071
Name: count, dtype: float64
 > Shuffle
 > Establecemos el batchsize


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 12), dtype=tf.float32, name=None))>

In [16]:
test_tfdataset = get_dataset(
    df=df_dataset_test,
    input_size=(224,224),
    batch_size=64,
    gray_scale=False,
    augmenter=False,
    test_set=True,
)

test_tfdataset

Number of instances per label: 
label
Unkown    794
Name: count, dtype: int64

Percentaje of instances per label: 
label
Unkown    1.0
Name: count, dtype: float64
 > Establecemos el batchsize


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

### Visualizamos los datasets preparado

No hacer si luego vamos a lanzar un entrenamiento, porque se iterar'an y al entrenar algunas muestras no estar;an y creo que va

In [ ]:
# TRAIN
batch = next(iter(train_tfdataset))
print
batches_card_np = batch[0].numpy()
y = batch[1].numpy()
images = list(iter(batches_card_np))

fig, axs = plt.subplots(math.ceil(len(images)/4), 4, figsize=(15, math.ceil(len(images)/4)*4))
axs = axs.ravel()

for i, image in enumerate(images):
    axs[i].imshow(image, cmap='gray')
    title = f"{dict_map_class_inverted[np.argmax(y[i])]}"
    axs[i].set_title(title)

In [ ]:
# VAL
batch = next(iter(valid_tfdataset))
print
batches_card_np = batch[0].numpy()
y = batch[1].numpy()
images = list(iter(batches_card_np))

fig, axs = plt.subplots(math.ceil(len(images)/4), 4, figsize=(15, math.ceil(len(images)/4)*4))
axs = axs.ravel()

for i, image in enumerate(images):
    axs[i].imshow(image, cmap='gray')
    title = f"{dict_map_class_inverted[np.argmax(y[i])]}"
    axs[i].set_title(title)

In [ ]:
# TEST
batch = next(iter(test_tfdataset))
print
batches_card_np = batch[0].numpy()
y = batch[1].numpy()
images = list(iter(batches_card_np))

fig, axs = plt.subplots(math.ceil(len(images)/4), 4, figsize=(15, math.ceil(len(images)/4)*4))
axs = axs.ravel()

for i, image in enumerate(images):
    axs[i].imshow(image, cmap='gray')
    title = f"{dict_map_class_inverted[np.argmax(y[i])]}"
    axs[i].set_title(title)

## **Prueba entrenamiento**

In [17]:
# Calculamos los pesos de cada clase debido al desbalanceo de las clases
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(df_dataset_train['label'].values),
                                     y=df_dataset_train['label'].values
)

class_weight_dict = dict(enumerate(class_weights))

class_weight_dict

{0: 0.6045251752708731,
 1: 0.6478825136612022,
 2: 0.7673948220064725,
 3: 0.7984006734006734,
 4: 0.8320175438596491,
 5: 1.0133547008547008,
 6: 1.0265151515151516,
 7: 1.3806404657933042,
 8: 1.5055555555555555,
 9: 1.7182971014492754,
 10: 1.7964015151515151,
 11: 1.7964015151515151}

In [19]:
# Cargamos el modelo de prueba
model = load_efficienNetV2(num_classes=12)
model.summary()

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(0.0001),
    metrics = ['acc']) 
                #tfam.F1Score(num_classes=5, name='f1_weig', threshold=0.5)])

monitoring_metric = 'val_acc'

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor = monitoring_metric, 
        mode = 'min', 
        patience = 20, 
        verbose=1,
        restore_best_weights=False),
    tf.keras.callbacks.ModelCheckpoint(
        filepath = 'models/CNN_source',
        monitor = monitoring_metric,
        mode = 'max',
        save_best_only = True,
        verbose = 1),
    ]

# Entrenamos el modelo
history = model.fit(train_tfdataset.repeat(),
        validation_data=valid_tfdataset,
        #class_weight = class_weights,
        epochs = 100,
        steps_per_epoch=100,
        callbacks = callbacks,
        verbose=1)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 224, 224, 3)          0         ['rescaling_1[0][0]']         
 tion)                                                                                            
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 32)         864       ['normalization_1[0][0]'

2023-11-16 13:29:52.553268: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


100/100 [==============================] - ETA: 0s - loss: 2.9921 - acc: 0.3129

2023-11-16 13:30:38.837277: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 1073741824 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:30:38.837321: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 1073741824
2023-11-16 13:30:38.961204: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 966367744 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:30:38.961246: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 966367744
2023-11-16 13:30:39.092615: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 869731072 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:30:39.092665: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 869731072


Epoch 1: val_acc improved from -inf to 0.58787, saving model to models/CNN_source
INFO:tensorflow:Assets written to: models/CNN_source/assets


INFO:tensorflow:Assets written to: models/CNN_source/assets


100/100 [==============================] - 87s 620ms/step - loss: 2.9921 - acc: 0.3129 - val_loss: 1.2003 - val_acc: 0.5879
Epoch 2/100
100/100 [==============================] - ETA: 0s - loss: 1.1422 - acc: 0.6368

2023-11-16 13:31:53.426863: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 2147483648 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:31:53.426914: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 2147483648
2023-11-16 13:32:06.946962: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 2147483648 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:32:06.947011: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 2147483648
2023-11-16 13:32:20.466766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 2147483648 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:32:20.466812: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 214748

ResourceExhaustedError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/diz-wsl/anaconda3/envs/viu-gpu/lib/python3.9/asyncio/base_events

2023-11-16 13:32:47.906531: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 2147483648 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:32:47.906581: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 2147483648
2023-11-16 13:33:01.347755: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 2147483648 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:33:01.347803: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 2147483648
2023-11-16 13:33:24.867942: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:809] failed to alloc 2147483648 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-11-16 13:33:24.867983: W ./tensorflow/compiler/xla/stream_executor/device_host_allocator.h:52] could not allocate pinned host memory of size: 214748

In [ ]:
predictions = model.predict(test_tfdataset)